In [4]:
#!pip install fastai

^C


In [18]:
from flask import Flask, render_template, request, redirect, url_for
from werkzeug.utils import secure_filename
import os

# fixes a dependancy on PosixPath by load_learner
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

import random as rnd
from fastai import *
from fastai.vision.all import *

app = Flask(__name__)
if not os.path.isdir('UploadFolder'): os.mkdir('UploadFolder')
app.config['UPLOAD_FOLDER'] = './static/UploadFolder'

# These are the categories used in the model and the data
DATAPATH = "./static/data/"
CATEGORIES = ["art deco","baroque","gothic","roman","tudor"]
# Every category has at least 600 images
N_MAX = 20

sample = {}
 
learn = load_learner('export.pkl')

def format_example(imgpath,cat,prediction):
   return {"image":"/data/"+imgpath,
                           "category":cat,
                           "prediction":prediction[0],
                           "correct_class":"example_"+str(cat==prediction[0])}

def format_sample(imgpath,prediction):
   return {"image":"/UploadFolder/"+imgpath,
                           "prediction":prediction[0]}

def get_examples():
   examples = []
   for cat in CATEGORIES:
      n = rnd.choice(range(N_MAX))
      imgpath = cat+"/"+cat+str(n)+".png"
      prediction = learn.predict(DATAPATH+imgpath)

      examples.append(format_example(imgpath,cat,prediction))
   
   return examples


   
    


@app.route('/')
def home():
   global sample
   print(sample)
   examples = get_examples()
   return render_template('home.html',examples=examples,sample=sample)

@app.route("/testImage_post", methods=['post'])
def testImage_post():
   for img in os.listdir(app.config['UPLOAD_FOLDER']):
      os.remove(os.path.join(app.config['UPLOAD_FOLDER'],img))
   global sample
   # examples are manualy set for now, straight from the model? or select some to test and test them and than compare to name using regex?

   image = request.files['file']
   imgpath = secure_filename(image.filename)
   image.save(os.path.join(app.config['UPLOAD_FOLDER'], imgpath))

   #test the posted image with the model
   prediction = learn.predict(os.path.join(app.config['UPLOAD_FOLDER'], imgpath))

   sample = format_sample(imgpath,prediction)



   return redirect(url_for('home'))



if __name__ == '__main__':
   app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2021-12-11 14:29:50,647] ERROR in app: Exception on /testImage_post [POST]
Traceback (most recent call last):
  File "C:\Users\32492\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\32492\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\32492\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\32492\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\32492\AppData\Local\Temp/ipykernel_21852/1521862105.py", line 64, in testImage_post
    os.remove(img)
FileNo